# Libraries

In [1]:
# importing libraries
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
import pandas as pd
from networkx.algorithms.community import *
from sklearn.cluster import *

# Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset Reading

In [3]:
data_vil2 = pd.read_csv("/content/drive/MyDrive/Project/vil2.csv")
data_vil2.drop("Unnamed: 0",axis=1,inplace=True)
data_vil2

,level_0,level_1
0,0,1
1,0,2
2,0,3
3,0,4
4,0,6
...,...,...
6115,875,876
6116,876,872
6117,876,873
6118,876,874


In [4]:
# Install karateclub API
! pip install karateclub

In [5]:
from karateclub import DeepWalk

In [15]:
# -----------------------------------
# function to create graph
# -----------------------------------
def create_graph(dataset):

  # empty directed graph Gc
  Gc = nx.DiGraph()

  # populating graph
  for i,j in zip(dataset["level_0"],dataset["level_1"]):
      Gc.add_edge(i,j)
  
  # return graph
  return Gc


# -----------------------------------
# function to set status of all nodes of G to "inactive"
# -----------------------------------
def set_status_inactive(G):
    for i in G.nodes():
        G.nodes[i]['status'] = "inactive"

# -----------------------------------
# function to set status of list of nodes to active
# -----------------------------------
def set_status_active(G,l):
    for i in l:
        G.nodes[i]['status'] = "active"


# -----------------------------------
# function to set threshold of the nodes
# -----------------------------------
def set_threshold(G):
    random.seed(1)
    x = [random.uniform(0,1) for i in range(G.number_of_nodes())]
    for i,j in zip(G.nodes(),x):
        G.nodes[i]['threshold'] = j


# -----------------------------------
# function to set node influence
# -----------------------------------
def active_only(Gc,v):
    node_influence = 0
    status = Gc.nodes[v]["status"]
    if status == "active":
        node_influence = 1
    return node_influence

# -----------------------------------
# function to define aggregated influence on node u
# -----------------------------------
def peer_influence(Gc,u):
    
    count = 0
    pi = 0
    for v in Gc.predecessors(u):
        count += 1
        pi += (1 * active_only(Gc,v))
        
    if count != 0:
        pi = pi / count
    else:
        pi = 0
    
    return pi


# -----------------------------------
# function to calculate number of active nodes
# -----------------------------------
def count_active(G):
    count = 0
    for i in G.nodes():
       if G.nodes[i]["status"] == "active":
            count += 1
    return count

# -----------------------------------
# function to create node embeddings
# -----------------------------------
def community_detection_datamining(G,emb):
      num_clusters = 10
      kmeans = KMeans(n_clusters=num_clusters, random_state=11).fit(emb)
      clusters = [[] for i in range(num_clusters)]
      for i,j in zip(emb,G.nodes()):
          val = kmeans.predict(i.reshape(1, -1))
          clusters[val[0]].append(j)
      
      # convert list to frozen set
      clusters = list(map(frozenset, clusters))
      return clusters

# -----------------------------------
# function to create node embeddings
# -----------------------------------
def deep_walk(G):
      # train model and generate embedding
      model = DeepWalk(walk_length=50, dimensions=12, window_size=4)
      mapping = dict(zip(G, range(0,G.number_of_nodes())))
      newG = nx.relabel_nodes(G, mapping)
      model.fit(newG)
      embedding = model.get_embedding()
      communities = community_detection_datamining(G,embedding)
      return communities # list of frozensets

# -----------------------------------
# find influential nodes using community structure
# -----------------------------------
def findInfluentialNodes(Gc,number_of_influential_nodes):

    # output = greedy_modularity_communities(Gc)
    output = deep_walk(Gc)

    num = number_of_influential_nodes

    mydict = {}
    for i in output:
      mydict[i] = len(i)

    influential_nodes = []


    for i in mydict.keys():
      if len(influential_nodes) < num:
        subgraph = nx.subgraph(Gc,i) 
        pr = nx.pagerank(subgraph)
        # lr = leader_rank(subgraph)
        # diction = {}
        # for i,j in zip(subgraph.nodes(),lr):
        #   diction[i] = j
        # pr = sorted(
        #   diction.items(),
        #   key = lambda kv: kv[1],reverse=True)
        
        pr = sorted(
          pr.items(),
          key = lambda kv: kv[1],reverse=True)
        influential_nodes.append(pr[0][0])
      else:
        return influential_nodes

    return influential_nodes




# -----------------------------------
# function to select set of p nodes of Gc as seed nodes based on some centrality
# -----------------------------------
def Centrality(Gc,p,centrality_measure):
        if centrality_measure == "degree":
            pr = nx.degree_centrality(Gc)
        elif centrality_measure == "closeness":
            pr = nx.closeness_centrality(Gc)
        elif centrality_measure == "betweeness":
            pr = nx.betweenness_centrality(Gc)
        elif centrality_measure == "pagerank":
            pr = nx.pagerank(Gc)
        elif centrality_measure == "katz":
            pr = nx.katz_centrality(Gc, 0.07)
        elif centrality_measure == "community":
            pr = findInfluentialNodes(Gc,p)
            print("pr:",pr)
            return pr
        n = Gc.number_of_nodes()
        n = p
        pr = {k: v for k, v in sorted(pr.items(), key=lambda item: item[1],reverse=True)}
        return list(pr.keys())[:n]

# -----------------------------------
# general threshold model for cascade
# -----------------------------------
def gt_model(Gc,p=10,imax=10,centrality_measure="pagerank"):
    
    AS = [0 for i in range(0,imax+1)]
    flag = 1
    
    # ----------------------------------------
    # Step 1
    # ----------------------------------------
    
    # set status of all nodes in Gc as inactive
    set_status_inactive(Gc)
    
    # initial set of active nodes
    AS[0] = Centrality(Gc,p,centrality_measure=centrality_measure)
    
    # set status of AS nodes to active
    set_status_active(Gc,AS[0])
    
    # ----------------------------------------
    # Step 2
    # ----------------------------------------
    for i in range(0,imax+1):
        
        print("-"*50)
        print("Value of i:",i)
        print("-"*50)
        
        print("At start of iteration:",i)
        print(f"Number of active nodes: {count_active(Gc)}/{Gc.number_of_nodes()}, {(count_active(Gc)/Gc.number_of_nodes()*100):.2f}% nodes")
        
        PS = [] # pending active set
        for u in Gc.nodes():
            if ((Gc.nodes[u]['status'] == "inactive") and (u not in PS)):
                score = peer_influence(Gc,u)
                if score > Gc.nodes[u]['threshold']:
                    PS.append(u)

        if i < imax: 
          AS[i+1] = AS[i] + PS
          AS[i+1] = list(set(sorted(AS[i+1])))
          
          if AS[i+1] == AS[i]:
              print("Converged at iteration:",i)
              flag = 0
              break
          
          set_status_active(Gc,AS[i+1])
          print("At end of iteration:",i)
          print(f"Number of active nodes: {count_active(Gc)}/{Gc.number_of_nodes()}, {(count_active(Gc)/Gc.number_of_nodes()*100):.2f}% nodes")
    
    if flag:
        print("Converged at iteration:",imax)

In [16]:
G2 = create_graph(data_vil2)
set_status_inactive(G2)
set_threshold(G2)
gt_model(G2,p=10,imax=10,centrality_measure="community") 

pr: [297, 730, 368, 485, 235, 51, 150, 98, 44, 32]
--------------------------------------------------
Value of i: 0
--------------------------------------------------
At start of iteration: 0
Number of active nodes: 10/876, 1.14% nodes
At end of iteration: 0
Number of active nodes: 29/876, 3.31% nodes
--------------------------------------------------
Value of i: 1
--------------------------------------------------
At start of iteration: 1
Number of active nodes: 29/876, 3.31% nodes
At end of iteration: 1
Number of active nodes: 53/876, 6.05% nodes
--------------------------------------------------
Value of i: 2
--------------------------------------------------
At start of iteration: 2
Number of active nodes: 53/876, 6.05% nodes
At end of iteration: 2
Number of active nodes: 84/876, 9.59% nodes
--------------------------------------------------
Value of i: 3
--------------------------------------------------
At start of iteration: 3
Number of active nodes: 84/876, 9.59% nodes
At end o

# Saving state

In [81]:
final_nodes = [297, 730, 368, 485, 235, 51, 150, 98, 44, 32]
final_nodes

[297, 730, 368, 485, 235, 51, 150, 98, 44, 32]

In [82]:
with open('/content/drive/MyDrive/Project/sna_1/models/final_nodes', 'wb') as files:
    pickle.dump(final_nodes, files)

In [83]:
with open('/content/drive/MyDrive/Project/sna_1/models/final_nodes' , 'rb') as f:
    final_nodes = pickle.load(f)

In [84]:
# -----------------------------------
# general threshold model for cascade
# -----------------------------------
def gt_model_save(Gc,p=10,imax=10,centrality_measure="pagerank"):
    
    AS = [0 for i in range(0,imax+1)]
    flag = 1
    
    # ----------------------------------------
    # Step 1
    # ----------------------------------------
    
    # set status of all nodes in Gc as inactive
    set_status_inactive(Gc)
    
    # initial set of active nodes
    AS[0] = final_nodes
    
    # set status of AS nodes to active
    set_status_active(Gc,AS[0])

    sourceFile = open('/content/drive/MyDrive/Project/sna_1/models/gt_model_final.txt', 'w')
    
    # ----------------------------------------
    # Step 2
    # ----------------------------------------
    for i in range(0,imax+1):
        
        print("-"*50, file = sourceFile)
        print("Value of i:",i, file = sourceFile)
        print("-"*50, file = sourceFile)
        
        print("At start of iteration:",i, file = sourceFile)
        print(f"Number of active nodes: {count_active(Gc)}/{Gc.number_of_nodes()}, {(count_active(Gc)/Gc.number_of_nodes()*100):.2f}% nodes", file = sourceFile)
        
        PS = [] # pending active set
        for u in Gc.nodes():
            if ((Gc.nodes[u]['status'] == "inactive") and (u not in PS)):
                score = peer_influence(Gc,u)
                if score > Gc.nodes[u]['threshold']:
                    PS.append(u)

        if i < imax: 
          AS[i+1] = AS[i] + PS
          AS[i+1] = list(set(sorted(AS[i+1])))
          
          if AS[i+1] == AS[i]:
              print("Converged at iteration:",i, file = sourceFile)
              sourceFile.close()
              flag = 0
              break
          
          set_status_active(Gc,AS[i+1])
          print("At end of iteration:",i, file = sourceFile)
          print(f"Number of active nodes: {count_active(Gc)}/{Gc.number_of_nodes()}, {(count_active(Gc)/Gc.number_of_nodes()*100):.2f}% nodes", file = sourceFile)
    
    if flag:
        print("Converged at iteration:",imax, file = sourceFile)
        sourceFile.close()

In [85]:
G2 = create_graph(data_vil2)
set_status_inactive(G2)
set_threshold(G2)
gt_model_save(G2,p=10,imax=10,centrality_measure="community")

In [86]:
with open('/content/drive/MyDrive/Project/sna_1/models/gt_model_final.txt', 'r') as f:
  print(f.read())

--------------------------------------------------
Value of i: 0
--------------------------------------------------
At start of iteration: 0
Number of active nodes: 10/876, 1.14% nodes
At end of iteration: 0
Number of active nodes: 29/876, 3.31% nodes
--------------------------------------------------
Value of i: 1
--------------------------------------------------
At start of iteration: 1
Number of active nodes: 29/876, 3.31% nodes
At end of iteration: 1
Number of active nodes: 53/876, 6.05% nodes
--------------------------------------------------
Value of i: 2
--------------------------------------------------
At start of iteration: 2
Number of active nodes: 53/876, 6.05% nodes
At end of iteration: 2
Number of active nodes: 84/876, 9.59% nodes
--------------------------------------------------
Value of i: 3
--------------------------------------------------
At start of iteration: 3
Number of active nodes: 84/876, 9.59% nodes
At end of iteration: 3
Number of active nodes: 111/876, 12.

## Function for leader rank

In [8]:
import copy

def leader_rank(cora):
  num_nodes = cora.number_of_nodes()
  num_edges = cora.number_of_edges()
  LeaderRank = np.ones(num_nodes + 1)
  LeaderRank[num_nodes] = 0

  newG = cora.copy()
  newG.add_node('0')
  for node in newG.nodes():
      newG.add_edge('0', node)

  error = 10000
  error_threshold = 2e-3
  step = 1
  max_steps = 300

  while True:
      tempLR = np.copy(LeaderRank)
      for i, node_i in enumerate(newG.nodes()):
          for j, node_j in enumerate(newG.neighbors(node_i)):
              tempLR[i] += LeaderRank[j] / newG.degree(node_j)

      error = np.sum(np.abs(tempLR - LeaderRank)) / np.sum(tempLR)
      print("Step:", step, ", error:", error)
      LeaderRank = np.copy(tempLR)
      if error <= error_threshold:
          print("Below Error Threshold")
          break
      step = step + 1
      
      if step > max_steps:
          print("Max Steps Reached")
          break

  LeaderRank = LeaderRank + LeaderRank[-1] / num_edges
  LeaderRank = LeaderRank[:-1] # Pop ground node
  LeaderRank = LeaderRank / np.sum(LeaderRank)
  return LeaderRank